In [1]:
from typing import Dict
from rich import print
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#custom imports
from Load import Load_Type, Load_Category, Load, Wall, Concrete, Soil

In [2]:
#Soil Properties
gamma_s = 19.0 # kPa/m
ka = 0.33
ko = 0
kp = 3.0
Q_sls = 300
Q_sls = 200
mu = 0.4

soil = Soil('soil', gamma_s, ka, ko, kp, Q_sls, Q_sls, mu)

# concrete properties
fc = 25 # MPa
Ec = 4500*(fc**0.5) # MPa
alpha1 = 0.91
beta1 = 0.81

C25 = Concrete('C25', fc)

In [4]:
# Geometry
# Wall
h = 1800 # mm
tw = 200 # mm
L = 1000 # mm

# Footing
tc = 300 # mm
toe = 250 # mm
b = 1500 # mm
D = 300 # mm
heel = b-toe-tw # mm

wall = Wall('wall1', h, tw, tc, toe, b, D, heel, C25)

In [5]:
# Loads
# Soil
q_soil = ka*gamma_s*((h+tc+D)/1000)*(L/1000) # kPa
q_eq = 0.25*gamma_s*((h+tc+D)/1000)*(L/1000) # kPa

# compaction
q_comp = 5 # kPa

# surcharge
W_dl = 2.4 # kPa
W_ll = 4.8 # kpa

# Wall Loads
P_dl = 100 # kN/m
P_sl = 200 # kN/m

wall.Add_Load(Load_Type.HYDROSTATIC, Load_Category.SOIL, 'soil', q_soil)
wall.Add_Load(Load_Type.SEISMIC, Load_Category.SEISMIC, 'eq', q_eq)

wall.Add_Load(Load_Type.SURCHARGE, Load_Category.COMPACTION, 'comp', q_comp)

wall.Add_Load(Load_Type.SURCHARGE, Load_Category.DEAD, 'dl', W_dl)
wall.Add_Load(Load_Type.SURCHARGE, Load_Category.LIVE, 'll', W_ll)

wall.Add_Load(Load_Type.WALL, Load_Category.DEAD, 'wall_dl', P_dl)
wall.Add_Load(Load_Type.WALL, Load_Category.SNOW, 'wall_sl', P_sl)

In [6]:
def Sum_Loads(loads: Dict, load_type: Load_Type)-> Dict:
    load_dict = {
        'Dead': 0,
        'Live': 0,
        'Storage': 0,
        'Snow': 0,
        'Seismic': 0,
        'Soil': 0,
        'Compaction': 0
    }
    for k, load in loads.items():
        if load.Type == load_type:
            if load.Category == Load_Category.DEAD:
                load_dict['Dead'] =+ load.Value
            if load.Category == Load_Category.LIVE:
                load_dict['Live'] =+ load.Value
            if load.Category == Load_Category.STORAGE_LIVE:
                load_dict['Storage'] =+ load.Value
            if load.Category == Load_Category.SNOW:
                load_dict['Snow'] =+ load.Value
            if load.Category == Load_Category.SEISMIC:
                load_dict['Seismic'] =+ load.Value
            if load.Category == Load_Category.SOIL:
                load_dict['Soil'] =+ load.Value
            if load.Category == Load_Category.COMPACTION:
                load_dict['Compaction'] =+ load.Value
    return load_dict

def SLS_Load_Combos(load_dict: Dict) -> Dict:
    combo_dict = {}
    combo_dict['1.0DL'] = load_dict['Dead'] * 1.0
    combo_dict['1.0DL+1.0LL+1.0L+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Live'] * 1.0 + load_dict['Snow'] * 1.0 + 1.0 * load_dict['Soil']
    combo_dict['1.0DL+1.0EQ+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Seismic'] * 1.0 + 1.0 * load_dict['Soil']
    combo_dict['1.0DL+1.0EQ++0.5LL+0.25SL+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Seismic'] * 1.0 + load_dict['Live'] * 0.5 + load_dict['Snow'] * 0.25 + 1.0 * load_dict['Soil']
    return combo_dict

def ULS_Load_Combos(load_dict: Dict) -> Dict:
    combo_dict = {}
    combo_dict['1.4DL+1.5Compaction'] = load_dict['Dead'] * 1.4 +load_dict['Compaction'] * 1.5
    combo_dict['1.4DL'] = load_dict['Dead'] * 1.4
    combo_dict['1.25DL+1.5LL+1.0SL+1.5H'] = load_dict['Dead'] * 1.25 + load_dict['Live'] * 1.5 + load_dict['Snow'] * 1.0 + 1.5 * load_dict['Soil']
    combo_dict['1.25DL+1.5SL+1.0LL++1.5H'] = load_dict['Dead'] * 1.25 + load_dict['Snow'] * 1.5 + load_dict['Live'] * 1.0+ 1.5 * load_dict['Soil']
    combo_dict['1.25DL+1.5LL+1.5H'] = load_dict['Dead'] * 1.25 + load_dict['Live'] * 1.5 + 1.5 * load_dict['Soil']
    combo_dict['1.25DL+1.5SL+1.5H'] = load_dict['Dead'] * 1.25 + load_dict['Snow'] * 1.5 + 1.5 * load_dict['Soil']
    combo_dict['1.0DL+1.0EQ+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Seismic'] * 1.0 + 1.0 * load_dict['Soil']
    combo_dict['1.0DL+1.0EQ+0.5LL+0.25SL+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Seismic'] * 1.0 + load_dict['Live'] * 0.5 + load_dict['Snow'] * 0.25 + 1.0 * load_dict['Soil']
    # combo_df = pd.DataFrame(combo_dict)
    return combo_dict

In [8]:
hydro_loads = {}
sur_loads = {}
wall_loads = {}
eq_loads = {}

hydro_loads = Sum_Loads(wall.Loads, Load_Type.HYDROSTATIC)
sur_loads = Sum_Loads(wall.Loads, Load_Type.SURCHARGE)
wall_loads = Sum_Loads(wall.Loads, Load_Type.WALL)
eq_loads = Sum_Loads(wall.Loads, Load_Type.SEISMIC)

hydro_uls_combos = {}
sur_uls_combos = {}
wall_uls_combos = {}
eq_combos = {}

hydro_uls_combos = ULS_Load_Combos(hydro_loads)
sur_uls_combos = ULS_Load_Combos(sur_loads)
wall_uls_combos = ULS_Load_Combos(wall_loads)
eq_combos = ULS_Load_Combos(eq_loads)

hydro_sls_combos = {}
sur_sls_combos = {}
wall_sls_combos = {}

hydro_sls_combos = SLS_Load_Combos(hydro_loads)
sur_sls_combos = SLS_Load_Combos(sur_loads)
wall_sls_combos = SLS_Load_Combos(wall_loads)

for k, v in eq_combos.items():
    pass
    #print(f"{k} -> {v}")

In [17]:
def Hydro(wall: Wall, load: float):
    h = (wall.Height + wall.Toe_Cover + wall.Footing_Thickness)/1000
    P = load*h/2
    x = h/3
    M = P*x
    return {'P': P, 'x': x, 'M': M}

def Surcharge(wall: Wall, load: float):
    h = (wall.Height + wall.Toe_Cover + wall.Footing_Thickness)/1000
    P = load*h
    x = h/2
    M = P*x
    return {'P': P, 'x': x, 'M': M}

def Wall(wall: Wall, load: float):
    P = load
    x = (wall.Wall_Thickness/2 + wall.Toe_length)/1000
    M = P*x
    return {'P': P, 'x': x, 'M': M}

def Seismic(wall: Wall, load: float):
    h = (wall.Height + wall.Toe_Cover + wall.Footing_Thickness)/1000
    P = load*h/2
    x = 2*h/3
    M = P*x
    return {'P': P, 'x': x, 'M': M}

def Analyze_Wall(load_type: Load_Type, wall: Wall, combos: Dict):
    resultants = {}

    for combo, load in combos.items():
        if load_type == Load_Type.HYDROSTATIC:
            resultants[combo] = Hydro(wall, load)
        if load_type == Load_Type.SURCHARGE:
            resultants[combo] = Surcharge(wall, load)
        if load_type == Load_Type.WALL:
            resultants[combo] = Wall(wall, load)
        if load_type == Load_Type.SEISMIC:
            resultants[combo] = Seismic(wall, load)
    
    return resultants
    


hydro_sls_res = Analyze_Wall(Load_Type.HYDROSTATIC, wall, hydro_sls_combos)
sur_sls_res = Analyze_Wall(Load_Type.SURCHARGE, wall, sur_sls_combos)
wall_sls_res = Analyze_Wall(Load_Type.WALL, wall, wall_sls_combos)
eq_res = Analyze_Wall(Load_Type.SEISMIC, wall, eq_combos)

In [ ]:
geo_loads = {}
struct_loads = {}



In [ ]:
   
# # Construction
# q_soil
# q_comp
# blend = True
# if blend:
#     x = q_comp/(ka*gamma_s*(L/1000))*1000
#     if x > (h+tc+D):
#         x = 0
# P_comp = q_comp*(h+tc+D)/1000*L/1000
# P_soil = q_soil*x/2000

# V = P_comp + P_soil
# M = P_comp*(h+tc+D)/2000 + P_soil * (h+tc+D)/3000
# geo_loads['LC1'] = [V,M]

# # Service
# q_soil
# q_comp
# blend = True
# if blend:
#     x = q_comp/(ka*gamma_s*(L/1000))*1000
#     if x > (h+tc+D):
#         x = 0
# P_comp = q_comp*(h+tc+D)/1000*L/1000
# P_soil = q_soil*x/2000

# P = P_comp + P_soil
# M = P_comp*(h+tc+D)/2000 + P_soil * (h+tc+D)/3000
# loads['LC1'] = [P,M]